# Intargram Crawler V 1.1
- 나무위키에 등록된 인물의 인스타그램 URL 수집 
    - __동명이인 이슈 발생 (요 수정)__
- 수집된 URL에 차례로 접근하여 필요한 정보 수집
    - 팔로워 수 (O)
    - 팔로잉 수 (O)
    - 게시글 수 (O)
    - 좋아요 수 (O)
        - 평균 좋아요 수
    - 업로드 일 수 (O)
        - 업로드 간격
        - 가장 최근 업로드 일

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
from tqdm import trange
from urllib import parse
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import lxml
import time

# 검색어 입력

In [ ]:
target_txt = ['김연아', '장성규', '김우빈', '이순재', '손흥민', '안재홍',  '유재석', '정준하', '정형돈',  '장도연']

# 나무위키

- 각 인물 정보 페이지에서 인스타그램 링크 수집

In [ ]:
# 인물의 인스타그램 정보 추출
def url_collector(for_same = False, retry = False):
    
    instagram = []
    dt = {}
    # 인물의 이름을 차례로받아 url을 완성
    for name in target_txt:
        ans = False
        if for_same == False:
            url_namu = 'https://namu.wiki/w/{}'.format(name)
        else:
            print(for_same)
            url_namu = for_same
        
        # 해당 인물정보 페이지의 소스를 분석
        r = requests.get(url_namu)
        bs = BeautifulSoup(r.text, 'html.parser')
        
        # 해당 페이지 소스에 모든 <table> tag를 가져옴
        table = bs.find_all('table')
        
        # 많은 table 중 인물정보 table을 특정
        for i in table:
            if '출생' in str(i):
                target_table = i
                break
        
        # 해당 table의 모든 row 조회
        rows = target_table.find_all('tr')
        
        # 마지막 row = SNS link
        target_row = rows[-1].find_all('a')
        
        # SNS row에서 instagram url 조회, 성공 시 반환
        for x in target_row:
            if 'instagram' in x.get('href'):
                ans = x.get('href')
                instagram.append(ans)
                print(ans, name)
                dt[name] = ans
                
    return dt ### 이름도 반환
instagram = url_collector()
instagram

In [ ]:
# 동명이인 이슈 발생 시 올바른 나무위키 인물정보 페이지 후보들을 리턴하는 함수
def same_name(name):
    serch_url = 'https://namu.wiki/Search?target=title_content&q={}&namespace='.format(name)
    namu = 'https://namu.wiki'
    full = []
    re = requests.get(serch_url)
    bs_same = BeautifulSoup(re.text, 'html.parser')
    section = bs_same.find('section')
    rows = section.find_all('div')
    for row in rows:
        for category in ['(배우)', '(가수)', '(모델)', '(방송인)', '(아나운서)', '(코미디언)']:
            if category in str(row):
                query = row.find('a').get('href')
                if len(query.split('/')) != 3:
                    tmp =query.split('/')
                    query = '/' + tmp[1] + '/' + tmp[2]
            
                full_url = [namu + query, category.replace('(', '').replace(')', '')]
                full.append(full_url)
    
    if full == []:
        full = [[namu + '/w/' + name]]
    return full
same_name('한혜진')#[0][0]

# 인스타그램
- 각 인물의 인스타그램 타임라인에서 필요한 정보 수집
- 필요 정보
    - 팔로워
    - 팔로잉
    - 게시글
    - 평균 좋아요 수
    - 평균 댓글 수
    - 평균 업로드 일 간격
    - 가장 최근 활동 일 (게시글 기준)
        - 현재 - 가장 최근 게시글 게시일(return : 일 수)
            

# 각 게시물 필요 정보 수집
- 게시일
- 좋아요

In [ ]:
options = webdriver.ChromeOptions()
options.headless = False
#options.add_argument("window-size=1920x1080") # 가상화면 크기
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36")

# 브라우저 기동
driver = webdriver.Chrome('./chromedriver', options = options)

# 정보 입력
id = '01089409078'
pw = '!high48top'

# 클립보드에 input을 복사한 뒤
# 해당 내용을 actionChain을 이용해 로그인 폼에 붙여넣기
driver.get('https://www.instagram.com/accounts/login')
# 모든 동작마다 크롬브라우저가 준비될 때 까지 최대 3초씩 대기
driver.implicitly_wait(3)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys(id)
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys(pw)
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
time.sleep(3)

driver.maximize_window()
driver.find_element_by_xpath('/html/body/div[1]/section/main/div/div/div/div/button').click()
time.sleep(0.5)
driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()


print('로그인 완료')

In [ ]:
total = pd.DataFrame()
for people, account in instagram.items():

    # 각 인물 인스타그램 URL 접근
    driver.get(account)
    time.sleep(1)
    # 닉네임 추출
    nick = account.split('/')[-1]

    # 계정 URL 끝이 '/'인 경우
    if nick == '':
        nick = account.split('/')[-2]

    # 게시글 수 추출    
    post = driver.find_element_by_xpath('/html/body/div[1]/section/main/div/header/section/ul/li[1]/span/span')\
    .text.strip()

    # 팔로워 수
    followers = driver.find_element_by_xpath('/html/body/div[1]/section/main/div/header/section/ul/li[2]/a/span')\
    .text.strip()
    followers = followers.replace('백만', '*1000000').replace('천', '*1000').replace('명', '').split('*')
    if len(followers) == 2:
        followers = int(float(followers[0]) * int(followers[1]))
    else:
        followers = int(followers[0])
        
    # 팔로잉 수
    followings = driver.find_element_by_xpath('/html/body/div[1]/section/main/div/header/section/ul/li[3]/a/span')\
    .text.strip()
    followings = followings.replace('백만', '*1000000').replace('천', '*1000').replace('명', '').split('*')
    if len(followings) == 2:
        followings = int(float(followings)[0]) * int(followings[1]) 
    else:
        followings = int(followings[0])
    
    # 데이터프레임 화
    total = total.append(pd.DataFrame({'Name' : [people], 'Nickname' : [nick], \
                        'Followers' : [format(followers, ',')], 'Followings' : [followings], 'Posts' : [post]}))
    
    PostInfo = pd.DataFrame()

    # 첫 번째 게시물 선택
    driver.find_element_by_xpath('/html/body/div[1]/section/main/div/div[2]/article/div[1]/div/div[1]/div[1]').click()
    
    # 게시물 끝까지 넘기기
    for PageNo in range(int(post)):
        # 게시일 수집
        date = driver.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/div[3]/div[2]/a/time').get_attribute('title')
        
        # 좋아요 수집
        try: # 사진
            like = driver.find_element_by_xpath("/html/body/div[4]/div[2]/div/article/div[3]/section[2]/div/div/button/span").text.replace(' ', '')
        except: # 동영상 
            like = np.NaN
        
        # 페이지 넘기기
        if PageNo == 0:
            driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/div/a').click()
            time.sleep(1)
        elif PageNo < (int(post) - 1): 
            driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/div/a[2]').click()
            time.sleep(1)
            
        # 데이터프레임 화
        PostInfo = PostInfo.append(pd.DataFrame({"Name" : [people], "PostDate" : [str(date).strip()], "PostLike" : [str(like)]}))
        
    break
PostInfo = PostInfo.reset_index(drop = True)
# 게시일 datetime화
PostInfo['Datetime'] = PostInfo['PostDate'].apply(lambda x: datetime.strptime(x, "%Y년 %m월 %d일"))
# 게시일 간격 변수 생성
PostInfo['PostInterval'] = PostInfo['Datetime'].diff(-1)
PostInfo['PostInterval'] = PostInfo['PostInterval'].apply(lambda x: x.days)
total = total.reset_index(drop = True)

In [ ]:
total

In [ ]:
PostInfo['PostInterval'][0].days

In [ ]:
total.to_csv('./Export/Instagram_Profile.csv', index = False, encoding = 'utf8')
PostInfo.to_csv('./Export/Instagram_Post_Info.csv', index = False, encoding = 'utf8' )

---

In [ ]:
PostInfo = pd.DataFrame()
for tile in tile_link:
    for t in tile:
        driver.get(t)
        time.sleep(1)
        date = driver.find_element_by_xpath("/html/body/div[1]/section/main/div/div[1]/article/div[3]/div[2]/a/time").get_attribute('title')
        try:
            like = driver.find_element_by_xpath("/html/body/div[1]/section/main/div/div[1]/article/div[3]/section[2]/div/div/button/span").text.replace(' ', '')
        except:
            like = np.NaN
        PostInfo = PostInfo.append(pd.DataFrame({"PostDate" : [str(date)], "PostLike" : [str(like)]}))
PostInfo = PostInfo.reset_index(drop = True)

In [ ]:
driver.close()

In [ ]:
PostInfo = pd.DataFrame()
for people, account in instagram.items():
    
    driver.get('https://www.instagram.com/hm_son7/')
    time.sleep(3)
    
    driver.find_element_by_xpath('/html/body/div[1]/section/main/div/div[2]/article/div[1]/div/div[1]/div[1]').click()
    # 게시물 끝까지 넘기기
    for PageNo in range(92 - 1):
        # 게시일 수집
        date = driver.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/div[3]/div[2]/a/time').get_attribute('title')
        
        # 좋아요 수집
        try: # 사진
            like = driver.find_element_by_xpath("/html/body/div[4]/div[2]/div/article/div[3]/section[2]/div/div/button/span").text.replace(' ', '')
        except: # 동영상 
            like = np.NaN
        
        # 데이터프레임 화
        PostInfo = PostInfo.append(pd.DataFrame({"Name" : [people], "PostDate" : [str(date)], "PostLike" : [str(like)]}))
        
        # 페이지 넘기기
        if PageNo == 0:
            driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/div/a').click()
            time.sleep(1)
        else : 
            driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/div/a[2]').click()
            time.sleep(1)
        
    break
PostInfo = PostInfo.reset_index(drop = True)